In [9]:
import pandas as pd

# Carga y exploración de los archivos
file_path_games = '../data/processed/processed_steam_games.csv'
steam_games_df = pd.read_csv(file_path_games)

file_path_reviews = '../data/processed/processed_users_reviews.csv'
users_reviews_df = pd.read_csv(file_path_reviews)

file_path_user_items = '../data/processed/processed_users_items.csv'
users_items_df = pd.read_csv(file_path_user_items)

# Uniendo los datos de juegos con los de usuarios
steam_games_df.rename(columns={'id': 'item_id'}, inplace=True)
combined_df = pd.merge(users_items_df, steam_games_df, on='item_id', how='left')

# Combinando todos los datos en un solo DataFrame
df_complete = pd.merge(combined_df, users_reviews_df, on=['user_id', 'item_id'], how='left')

# Reemplaza los valores NaN de la columna sentiment con la opción por defecto 1: neutral
df_complete['sentiment'] = df_complete['sentiment'].fillna(1)

df_complete

,user_id,items_count,item_id,playtime_forever,genres,app_name,price,developer,release_year,sentiment
0,76561197970982479,277,10.0,6.0,Action,Counter-Strike,9.99,Valve,2000.0,1.0
1,76561197970982479,277,20.0,0.0,Action,Team Fortress Classic,4.99,Valve,1999.0,1.0
2,76561197970982479,277,30.0,7.0,Action,Day of Defeat,4.99,Valve,2003.0,1.0
3,76561197970982479,277,40.0,0.0,Action,Deathmatch Classic,4.99,Valve,2001.0,1.0
4,76561197970982479,277,50.0,0.0,Action,Half-Life: Opposing Force,4.99,Gearbox Software,1999.0,1.0
...,...,...,...,...,...,...,...,...,...,...
10913204,76561198329548331,7,521570.0,4.0,Free to Play,You Have 10 Seconds 2,Free To Play,Tamationgames,2016.0,1.0
10913205,76561198329548331,7,521570.0,4.0,Indie,You Have 10 Seconds 2,Free To Play,Tamationgames,2016.0,1.0
10913206,76561198329548331,7,519140.0,3.0,NaN,NaN,NaN,NaN,NaN,1.0
10913207,edward_tremethick,0,NaN,NaN,Action,Batman: Arkham City - Game of the Year Edition,19.99,"Rocksteady Studios,Feral Interactive (Mac)",2012.0,1.0


In [10]:
# Considerando que df_complete es el DataFrame actual
#df = df_complete[['user_id', 'item_id', 'playtime_forever', 'genres', 'app_name', 'release_year', 'sentiment']]

# Eliminación de duplicados
df_complete = df_complete.drop_duplicates()

# Manejo de valores NaN
df_complete = df_complete.dropna(subset=['genres', 'release_year', 'playtime_forever'])

In [11]:
# Guardamos el dataframe procesado para los análisis posteriores
output_file_path = '../data/processed/processed_dataset_full.csv'
df_complete.to_csv(output_file_path, index=False)

output_file_path

'../data/processed/processed_dataset_full.csv'